In [ ]:
%matplotlib inline

In [ ]:
import os

import mne
import numpy as np


# Preprocessing
The steps in EEG preprocessing:

* Highpass filter
* Lowpass filter
* Artefact rejection
    * Ocular artifacts (EOG)
    * Heartbeat artifacts (ECG)
    * Power line noise
    * Methods:
        * Thresholding
        * ICA
* (Rereference)
* Epoching
* Evokeds (i.e. averaging)


In [ ]:
sample_data_folder = mne.datasets.sample.data_path()
sample_data_raw_file = os.path.join(sample_data_folder, 'MEG', 'sample',
                                    'sample_audvis_filt-0-40_raw.fif')

In [ ]:
raw = mne.io.read_raw_fif(sample_data_raw_file, preload=True)

In [ ]:
raw.pick_types(meg=False, eeg=True, stim=True)

In [ ]:
# filter your data

Default filtering for the sample data is:
* highpass: 0.1 Hz
* lowpass: 40 Hz

You can see this by:
print(raw.info)

In [ ]:
# set up and fit the ICA
ica = mne.preprocessing.ICA(n_components=20, random_state=97, max_iter=800)
ica.fit(raw)
ica.exclude = [0, 2]  # details on how we picked these are omitted here
ica.plot_properties(raw, picks=ica.exclude);

In [ ]:
ica.plot_components();

In [ ]:
#Plotting the time series of the ICA.
ica.plot_sources(raw, show_scrollbars=False)

In [ ]:
orig_raw = raw.copy()
raw.load_data()
ica.apply(raw)

There are many options with ICA, see
https://mne.tools/stable/auto_tutorials/preprocessing/40_artifact_correction_ica.html
for a tutorial on ICA in MNE-python.

In [ ]:
# the stimulus channel on the Electra vector view is: STI 014
events = ???

In [ ]:
events = mne.find_events(raw)

In [ ]:
event_dict = {'auditory/left': 1,
              'auditory/right': 2,
              'visual/left': 3,
              'visual/right': 4,
              }

In [ ]:
reject_criteria = dict(eeg=150e-6)  # 150 µV


In [ ]:
conds_we_care_about = ['auditory/left', 'auditory/right',
                       'visual/left', 'visual/right']


In [ ]:
reject = dict(eeg=150e-6)
picks = mne.pick_types(raw.info, meg=False, eeg=True, eog=True)

epochs = mne.Epochs(raw, events, event_dict, -.2, 0.5, picks=picks,
                    baseline=None, reject=reject, preload=True)


# Comparing evokeds

To visually comparing different evoked objects it is a good idea to calculate
 a _difference  wave_, i.e. one condition minus the other condition. In
 MNE-python ```mne.combine_evokeds```can do that.

In [ ]:
# Change to your evokeds
difference_wave = mne.combine_evoked([Evoked_0, Evoked_2], weights=[1, -1])


See also:
https://mne.tools/stable/generated/mne.viz.plot_compare_evokeds.html